In [1]:
# Python 3 notebook for neural network

import numpy as np
import math
import statistics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.utils import to_categorical
from keras.utils import plot_model
import IPython.display
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression

import importlib
import sys
import pickle
import torch
from torch.autograd import Variable
import torch.nn as nn
from random import shuffle

# https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
# pnn = importlib.import_module("pytorch-neural-network") # Don't need this anymore, will delete later
we = importlib.import_module("word_embeddings")
import re

Using TensorFlow backend.


In [2]:
""" Global Parameters """
# Load some data
wordEmbedDict = we.getWordEmbeddingDict() # Load the dictionary

# Labels
top_20 = sorted(['AskReddit', 'leagueoflegends', 'nba', 'funny', 'pics', 'nfl', 'pcmasterrace', \
          'videos', 'news', 'todayilearned', 'DestinyTheGame', 'worldnews', 'soccer', \
          'DotA2', 'AdviceAnimals', 'WTF', 'GlobalOffensive', 'hockey', 'movies', 'SquaredCircle'])

# Indices of our desired data
TRUE_LABEL = 8 # Index of the true label, hard coded
BODY_INDEX = 17 # Index of the reddit comment, hard coded

# Neural Network Parameters
NUM_SUBREDDITS = len(top_20)
NUM_FEATURES = 300 # length returned from embedding
NUM_EXAMPLES = 1000 # Arbitrary, choose however many we want to grab from the dataset
NUM_EPOCHS = 500 # plateus really fast... no need to run more epochs
NUM_HIDDEN_NEURONS = 40
NUM_LAYERS = 10
LEARNING_RATE = 0.01

SUBREDDIT = "leagueoflegends" # used for debugging,delete later
unparsed = "./data/condensed_dataset_SMALL.pkl" # will change this so we can just call the whole pkl set

# Encoder
encoder = LabelEncoder()
encoder.fit(top_20) # Encodes the NUM_SUBREDDITS subreddits

# Seed
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
""" """

' '

In [ ]:
'''Helper Functions'''

# Normalization of dataset

# Get mean of a column feature
def getMean(column):
    sum = 0
    n = len(column)
    for i in range(n):
        sum += column.iloc[i]
    mean = sum / float(n)
    return mean

def getVariance(column, mean):
    squareMeanSum = 0
    n = len(column)
    for i in range(n):
        squareMeanSum += (column.iloc[i] - mean)**2
    var = math.sqrt(squareMeanSum / float(n))
    return var

def normalizeSet(set):
    numRow = len(set.index)
    numCol = len(set.columns)
    for col in range(numCol):
        column = set.iloc[:,col]
        mean = getMean(column)
        var = getVariance(column, mean)
        
        for row in range(numRow):
            set.iloc[row, col] = float(set.iloc[row, col] - mean) / var
    return set

# @param dir: string, directory of pickle data
# @return dataset: unpickled dataset
def loadPickleData(dir):
    with open(dir, 'rb') as f:
        dataset = pickle.load(f)
    return dataset

# Returns [X, Y] with m examples
def loadData(pickleDir, m):
    pickle = loadPickleData(pickleDir)
    return vectorizeDataSet(pickle, m)

def stripNonAlpha(word):
    word = re.sub(r'\W+', '', word)
    return word

def vectorizeWord(word):
    word = stripNonAlpha(word)
    keyset = wordEmbedDict.keys() # words in the dictionary
    zeroVec = np.zeros((1, NUM_FEATURES))
    vWord = pd.DataFrame(zeroVec)
    
    if word in keyset:
        vWord = pd.DataFrame(wordEmbedDict[word]).transpose()
    return vWord # returns zero vector if the word is not in the dictionary

def vectorizeComment(body):
    vComment = np.zeros((1, NUM_FEATURES))
    vComment = pd.DataFrame(vComment)
    words = body.split()
    
    numWords = 0
    for word in words:
        vWord = vectorizeWord(word)
        numWords += 1
        vComment = vComment.add(vWord)
    
    for i in range(NUM_FEATURES):
        print("vComment number: ", vComment.iloc[1,i])
        print(type(vComment.iloc[1,i]))
        vComment.iloc[1, i] /= numWords
        
    
    return vComment

# Encodes a subreddit string into an unrolled one-hot pandas vector
def oneHotEncode(subreddit):
    #   https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
    encoded_Y = encoder.transform([subreddit])[0] # get the integer category
    
    oneHot = [0 for _ in range(NUM_SUBREDDITS)]
    oneHot[encoded_Y] = 1
    pandasOneHot = pd.DataFrame(oneHot)
    
    return pandasOneHot.transpose()
    
    
#TODO: check if seed is bad??
#TODO: limit number of features 
def vectorizeDataSet(data, m):
    data = pd.DataFrame(data)
    data = data.sample(frac=1, random_state=seed).reset_index(drop=True) # Shuffles data
    comments = data.pop(BODY_INDEX)
    true_labels = data.pop(TRUE_LABEL)
    
    unrollComment = comments[0]
    X = vectorizeComment(unrollComment)
    firstSubreddit = true_labels[0]
    Y = oneHotEncode(firstSubreddit)

    # For each example in old data set, get the actual comment and featurize it into X
    # Also get unrolled true label
    for i in range(1, m):
        comment = comments[i]
        example = vectorizeComment(comment)
        subreddit = true_labels[i]
        oneHot = oneHotEncode(subreddit)
        
        X = pd.concat([X, example])
        Y = pd.concat([Y, oneHot])
    return pd.concat([X, Y], axis=1)

In [ ]:
# Split dataset
#https://stackoverflow.com/questions/38250710/how-to-split-data-into-3-sets-train-validation-and-test/38251213#38251213
def train_validate_test_split(df, train_percent=.9, validate_percent=.05, seed=seed):
    m = len(df.index)
    
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[:train_end]
    validate = df.iloc[train_end:validate_end]
    test = df.loc[validate_end:]
    return train, validate, test

In [ ]:
%%time
df = loadData(unparsed, NUM_EXAMPLES)
train, validate, test = train_validate_test_split(df)

train_labels = train.iloc[:, NUM_FEATURES:]
validate_labels = validate.iloc[:, NUM_FEATURES:]
test_labels = test.iloc[:, NUM_FEATURES:]

train = train.iloc[:, :NUM_FEATURES]
validate = validate.iloc[:, :NUM_FEATURES]
test = test.iloc[:, :NUM_FEATURES]

# normed_train = normalizeSet(train)
# normed_validate = normalizeSet(validate)
# normed_test = normalizeSet(test)

In [ ]:
# %%time
# # Save the dataframe as a pickle file to be read later
# df.to_pickle("./data/pandas-pickle-small.pkl")

# check if potentially overwriting data with the same feature everytime
unpickled = d.read_pickle("./data/pandas.picked")
unpickled.tail()

In [ ]:
%%time
# Check softmax output (final layer output)
# Check gradients inside the keras model built in function
# Neural network function
def build_nn(hidden_layer_sizes):
    model = keras.Sequential()
    
    # Input layer
    model.add(layers.Dense(hidden_layer_sizes[0], input_dim=NUM_FEATURES, activation='relu'))
    
#      https://datascience.stackexchange.com/questions/19407/keras-built-in-multi-layer-shortcut
#     Hidden layers
    for size in hidden_layer_sizes[1:]:
        model.add(layers.Dense(size, activation='relu'))
    
    # Output layer
    model.add(layers.Dense(NUM_SUBREDDITS, activation='softmax'))
    
    # Optimizer. Can change this to whatever we want
    optimizer = tf.keras.optimizers.Adam(lr=LEARNING_RATE)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', 'categorical_crossentropy'])
    return model

In [ ]:
%%time
# Build the model
# Features might be an issue, pick different subreddits
# potentially try out CNN with matrix word embedding without combined word embedding
sizes_list = [NUM_HIDDEN_NEURONS for i in range(NUM_LAYERS)]
nn_model = build_nn(sizes_list)
nn_model.summary()

In [ ]:
%%time
# Buggy. Loss goes constant after a while :(
# Get neural network history
# Change batch size (inspect what happens with each ITERATION, not EPOCH)
# History is the progress of our neural network, will be used to plot cost functions
# nn_history = KerasClassifier(build_fn=nn_model, epochs=NUM_EPOCHS, verbose=2) doesnt work :(
nn_history = nn_model.fit(train, train_labels, epochs=NUM_EPOCHS, verbose=1,
         validation_data=(validate, validate_labels))

In [ ]:
# Plot metrics
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Cross Entropy Error')
    plt.plot(hist['epoch'], hist['categorical_crossentropy'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_categorical_crossentropy'],
           label = 'Val Error')
    plt.ylim([1,20])
    plt.legend()
    plt.show()
    
plot_history(nn_history)

In [ ]:
# See accuracy
hist = pd.DataFrame(nn_history.history)
accuracy_vec = hist.pop("acc")
finalAcc = accuracy_vec[len(accuracy_vec) - 1]
print("Final accuracy: {}%".format(finalAcc*100))

In [ ]:
%%time
# Logistic Regression Model
df2 = pd.read_pickle("./data/pandas-pickle-small.pkl")
df2 = df2.sample(frac=1, random_state=2).reset_index(drop=True) # Shuffles data
# # print("This is our X: ", df2.iloc[:, ]
numTrain = 1000
numTest = 500

X = df2.iloc[0:numTrain, :NUM_FEATURES]
X_array = X.values

y = df2.iloc[0:numTrain, NUM_FEATURES:]
y_array = y.values

y_array_rows = y_array.shape[0]

y_integer_classes = np.array([])
for i in range(y_array_rows):
    row = y_array[i]
    index = np.where(row==1)[0][0]
    y_integer_classes = np.append(y_integer_classes, int(index))

testX = df2.iloc[numTrain:numTrain+numTest, :NUM_FEATURES]
testX_array = testX.values
testY = df2.iloc[numTrain:numTrain+numTest, NUM_FEATURES:]
testY_array = testY.values

testy_array_rows = testY_array.shape[0]
testy_integer_classes = np.array([])
for i in range(testy_array_rows):
    row = testY_array[i]
    index = np.where(row==1)[0][0]
    testy_integer_classes = np.append(testy_integer_classes, int(index))

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
print("Prediction results: \n")
predictions = OneVsRestClassifier(LinearSVC(random_state=0, max_iter=5000)).fit(X_array, y_integer_classes).predict(testX_array)
print(predictions)
print("True results: \n")
print(testy_integer_classes)


# Expects list
def accuracy(prediction, true_label):
    count = 0
    total = len(predictions)
    for i in range(total):
        if predictions[i] == true_label[i]:
            count += 1
    return float(count) / total

pred_list = predictions.tolist()
testy_list = testy_integer_classes.tolist()
print("SVM accuracy: {}%".format(accuracy(pred_list, testy_list)*100))
        